# Módulo de preprocesamiento 
Ammi Beltrán & Fernanda Borja

## Importado de librerias

In [62]:
# ! pip install -U scikit-learn

     ---------------------------------------- 9.2/9.2 MB 25.7 MB/s eta 0:00:00
     ---------------------------------------- 302.2/302.2 kB ? eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\TheSy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [64]:
import os
import mne
import glob
#
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

import copy

Se crea el path de data

In [27]:
if not os.path.exists("data"):
    os.makedirs("data")
    print("Data directory created :D")

Dirección del dataset

In [3]:
# EDFDIR = "D:\\OneDrive\\OneDrive - Universidad de Chile\\Semestre X\\Inteligencia\\Proyecto\\dataset\\tuh_eeg"
EDFDIR = "c:\\Users\\TheSy\\Desktop\\tuh_eeg"
files = glob.glob(EDFDIR + '/**/*.edf', recursive=True)

## Preprocesamiento

In [51]:
# Usamos MNE
data = mne.io.read_raw_edf(files[0])
raw_data = data.get_data()
info = data.info
channels = data.ch_names

Extracting EDF parameters from c:\Users\TheSy\Desktop\tuh_eeg\aaaaaaaa\s001_2015_12_30\01_tcp_ar\aaaaaaaa_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


### Funciones
* Una vez funcione mover a .py y traer como librería 

Seleccion de canales

In [ ]:
def channel_select(data, channels):
    '''
    Selects channels from array 
    '''
    extracted = data.pick(channels, exclude="bads")
    return extracted

Clipeo

In [ ]:
def clip(data, channels,max= 500e-6):
    def cliper(array):
        for i in range(len(array)):
            if abs(array[i]) > max:
                array[i] = math.copysign(max,array[i])
        return array
    data.apply_function(cliper, picks=channels, channel_wise= True)

Filtrado

In [ ]:
def eeg_filter(data, lfreq = 0.3, hfreq= 80):
    '''
    
    '''
    data_copy = copy.copy(data)
    filtered = data_copy.filter(l_freq = lfreq,
                                h_freq = hfreq,
                                method = "iir"
                                )
    return filtered

Corte de primero minuto y de max

In [ ]:
def temporal_crop(data, tin = 60, tfin = 12*60):
    ''' 
    Cut the channels from the second "tin" to "tfin"
    '''
    data_copy = copy.copy(data)
    croped = data_copy.crop(tmin = tin, tmax = min(tfin, int(data.times[-1])))
    return croped

Marcado de épocas

In [ ]:
# eventos
def get_epochs(data, channels, window = 20.0):
    ''' 
    window es la ventana de tiempo
    '''
    data_copy = copy.copy(data)
    # Create events
    events = mne.make_fixed_length_events(data_copy, duration = window, first_samp = True)
    # Divide accordingly
    picks = channels
    epochs = mne.Epochs(raw = data_copy, events = events, picks = picks, preload = True,
                        tmin = 0., tmax = window, baseline = None,
                        flat = dict(eeg = 1e-6))
    
    epochs.drop(-1,reason = "Unfixed duration")
    return epochs

Downsample

In [ ]:
def downsample(epoch, freq = 200):
    ''' 
    Downsamples the data given by a factor
    En nuestro caso, down corresponde a (frecuencia que queremos)/(frecuencia actual)
    '''
    down = epoch.resample(freq, npad = "auto")
    return down

Normalización

In [ ]:
def normalization(epochs):
    obj = mne.decoding.Scaler(info = epochs.info, scalings='mean')
    values = obj.fit_transform(epochs.get_data())
    return values

***

Union

In [ ]:
def preprocessing(path):
    ''' 
    Pipeline de procesamiento c:
    - Canales c:
    - Filtrado c:
    - Downsample c:
    - Crop temporal (primer minuto y a los 11 minutos) c:  
    - Segmentación c:
    - Normalización c:
    '''

    files = glob.glob(path + '/**/*.edf', recursive = True)
    # windows = np.empty() EVALUAR
    windows = []
    for i in range(0, len(files)):
        data = mne.io.read_raw_edf(files[i], preload=True)
        # raw_data = data.get_data()
        # info = data.info
        channels = data.ch_names[:21]
        # Pipeline
        # 
        # mapping = rename_channels(data)

        # data.rename_channels(data.info, mapping)
        # data.rename_channels(mapping)

        # try:
        channel_data = channel_select(data, channels)
        clip(channel_data,channels)
        # except:
        #     channel_data = channel_select(data, CHANNELS_LE)
        #     ch = CHANNELS_LE
        if(int(data.times[-1]) < 100):
            continue
        filtered = eeg_filter(channel_data)
        trimmed_data = temporal_crop(filtered)
        epochs = get_epochs(trimmed_data, channels)
        down_data = downsample(epochs)
        norm_data = normalization(down_data)
        #
        windows.append(norm_data)
    return windows

In [2]:
# patient = glob.glob(EDFDIR + '/**')
# patient

## Uso de funciones

In [69]:

win = preprocessing(EDFDIR)
win

***

In [67]:
# Channels we want

def rename_channels(data):
    # info = data.info
    original = data.ch_names
    new = copy.copy(original)
    res = {}
    for ch in new:
        
        isin = False
        for n in CHANNELS:
            if n in ch:
                res[ch] = n
                isin = True
                break

        if not isin:
            res[ch] = ch
            
    return res
